# 🗑️ Data Cleaning

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](<https://colab.research.google.com/>)

Remove null values and all the information other than the important ones.

# 1. ⚙️ Imports

Import the necessary libraries and packages.

In [1]:
import re
import pandas as pd

# Local Libraries
from data import data

# 2. 📁 Data

Get the 🇺🇸 US Presidential Elections by States data.

In [2]:
states_dataframe_path = data.get_dataset_path("rbs", "raw", "us_presidential_elections_by_states", 1)
candidate_dataframe_path = data.get_dataset_path("rbs", "raw", "us_presidential_elections_candidate", 1)

In [3]:
states_dataframe = pd.read_csv(states_dataframe_path)

print("The first five records of the states dataframe: \n")
states_dataframe.head()

The first five records of the states dataframe: 



,Year,1972,1976,1980,1984,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
0,National popular vote,Nixon,Carter,Reagan,Reagan,Bush,Clinton,Clinton,Gore,Bush,Obama,Obama,Clinton,Biden,Trump
1,Alabama,Nixon,Carter,Reagan,Reagan,Bush,Bush,Dole,Bush,Bush,McCain,Romney,Trump,Trump,Trump
2,Alaska,Nixon,Ford,Reagan,Reagan,Bush,Bush,Dole,Bush,Bush,McCain,Romney,Trump,Trump,Trump
3,Arizona,Nixon,Ford,Reagan,Reagan,Bush,Bush,Clinton,Bush,Bush,McCain,Romney,Trump,Biden,Trump
4,Arkansas,Nixon,Carter,Reagan,Reagan,Bush,Clinton,Clinton,Bush,Bush,McCain,Romney,Trump,Trump,Trump


In [4]:
candidate_dataframe = pd.read_csv(candidate_dataframe_path)

print("The first five records of the candidate dataframe: \n")
candidate_dataframe.head()

The first five records of the candidate dataframe: 



,Year,1972,1976,1980,1984,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
0,Democratic candidate,George McGovern,Jimmy Carter,Jimmy Carter,Walter Mondale,Michael Dukakis,Bill Clinton,Bill Clinton,Al Gore,John Kerry,Barack Obama,Barack Obama,Hillary Clinton,Joe Biden,Kamala Harris
1,Republican candidate,Richard Nixon,Gerald Ford,Ronald Reagan,Ronald Reagan,George H. W. Bush,George H. W. Bush,Bob Dole,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,Donald Trump,Donald Trump


# 3. 🧹 Cleaning Messy Data

⚓️ Let's check the count of null values in every column. ⤵️

## 3.1. 🪹 Drop Null Values

Drop all the rows with null values, as those won't add any value according to our end goal.

In [5]:
state_null_count = states_dataframe.isnull().sum().reset_index()
state_null_count.columns = ["Column", "Null Count"]

print("Count of null values in each column: \n")
state_null_count

Count of null values in each column: 



,Column,Null Count
0,Year,0
1,1972,1
2,1976,1
3,1980,1
4,1984,2
5,1988,3
6,1992,3
7,1996,3
8,2000,3
9,2004,3


In [6]:
print("Shape of the state dataframe: ", states_dataframe.shape)

Shape of the state dataframe:  (55, 15)


⚓️ Now, drop all the rows that have null data, as those don't contain the primary candidates of the winners. ⤵️

In [7]:
states_dataframe.dropna(inplace = True)

In [8]:
print("Shape of the state dataframe after dropping all the rows containing null values: ", states_dataframe.shape)

Shape of the state dataframe after dropping all the rows containing null values:  (52, 15)


## 3.2. 🥡 Remove Unnecessary Data

Remove the unnecessary part of each cell of the dataframe, as those don't contain information helpful for achieving our goal.

In [9]:
print("Let's check the value of the 21st row and 7th column: ", states_dataframe.iloc[20, 6])

Let's check the value of the 21st row and 7th column:  Clinton (at-large and ME-01)


⚓️ The `(at-large and ME-01)` part of `Clinton (at-large and ME-01)` is unnecessary for our end goal.
As we have some data like this, we have to remove those from the strings. ⤵️

⚓️ Function for removing the unnecessary parts of the strings.

In [10]:
def remove_unnecessary_strings(string: str) -> str:
    """
    Removes any substrings enclosed in parentheses (including the parentheses themselves).
    Parameters:⤵️
        - string (str): The input string potentially containing text in parentheses.
    Returns:
        - str: A cleaned string with all substrings in parentheses removed.
    """
    return re.sub(r"\s*\([^)]*\)", "", string)

⚓️ Remove all the unnecessary parts of the strings and create a new dataframe. ⤵️

In [11]:
clean_states_dataframe = states_dataframe.map(remove_unnecessary_strings)

In [12]:
print("The value of the 21st row and 7th column (after removing unnecessary parts): ", clean_states_dataframe.iloc[20, 6])

The value of the 21st row and 7th column (after removing unnecessary parts):  Clinton


## 3.3. 🥬 Last Name to Full Name

⚓️ Replace the short name with the full name of a candidate. ⤵️

In [13]:
def replace_with_full_name(name: str, democratic: str, republican: str) -> str:
    """
    Replace a short candidate name with the full name if it matches a Democratic or Republican candidate.
    Parameters:
        - name (str): A short name of the candidate.
        - democratic (str): The full name of the Democratic candidate for a specific year.
        - republican (str): The full name of the Republican candidate for the same year.
    Returns:
        - str: The full candidate name if the short name is a substring of either candidate;
             otherwise, returns the original cell value.
    """
    if isinstance(name, str):
        if name in democratic:
            return democratic
        elif name in republican:
            return republican

    return name

⚓️ Apply the replacement for each column. ⤵️

In [14]:
for column in candidate_dataframe.columns:
    democratic_candidate = candidate_dataframe.loc[0, column]
    republican_candidate = candidate_dataframe.loc[1, column]

    clean_states_dataframe[column] = clean_states_dataframe[column].apply(
        lambda cell: replace_with_full_name(cell, democratic_candidate, republican_candidate)
    )


In [15]:
print("After cleaning the dataframe: \n")
clean_states_dataframe.head()

After cleaning the dataframe: 



,Year,1972,1976,1980,1984,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
0,National popular vote,Richard Nixon,Jimmy Carter,Ronald Reagan,Ronald Reagan,George H. W. Bush,Bill Clinton,Bill Clinton,Al Gore,George W. Bush,Barack Obama,Barack Obama,Hillary Clinton,Joe Biden,Donald Trump
1,Alabama,Richard Nixon,Jimmy Carter,Ronald Reagan,Ronald Reagan,George H. W. Bush,George H. W. Bush,Bob Dole,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,Donald Trump,Donald Trump
2,Alaska,Richard Nixon,Gerald Ford,Ronald Reagan,Ronald Reagan,George H. W. Bush,George H. W. Bush,Bob Dole,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,Donald Trump,Donald Trump
3,Arizona,Richard Nixon,Gerald Ford,Ronald Reagan,Ronald Reagan,George H. W. Bush,George H. W. Bush,Bill Clinton,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,Joe Biden,Donald Trump
4,Arkansas,Richard Nixon,Jimmy Carter,Ronald Reagan,Ronald Reagan,George H. W. Bush,Bill Clinton,Bill Clinton,George W. Bush,George W. Bush,John McCain,Mitt Romney,Donald Trump,Donald Trump,Donald Trump


In [16]:
unique_counts = clean_states_dataframe.nunique().reset_index()
unique_counts.columns = ["Column", "Unique Count"]

print("Unique values in each column: \n")
print(unique_counts)

Unique values in each column: 

   Column  Unique Count
0    Year            52
1    1972             2
2    1976             2
3    1980             2
4    1984             2
5    1988             2
6    1992             2
7    1996             2
8    2000             2
9    2004             2
10   2008             2
11   2012             2
12   2016             2
13   2020             2
14   2024             2


# 4. 🌪️ Data Integration

Combine the state dataframe and candidate dataframe together to get a state-by-party database based on the year.

In [17]:
contents = []

for idx, row in clean_states_dataframe.iterrows():
    content = []

    for column in clean_states_dataframe.columns:
        name = row[column]
        democratic_candidate = candidate_dataframe.loc[0, column]
        republican_candidate = candidate_dataframe.loc[1, column]
        democratic_label = candidate_dataframe.iloc[0, 0].split()[0]
        republican_label = candidate_dataframe.iloc[1, 0].split()[0]

        if name in democratic_candidate:
            content.append(democratic_label)
        elif name in republican_candidate:
            content.append(republican_label)
        else:
            content.append(name)

    contents.append(content)

print("1st row of the state-party list: ", contents[0])

1st row of the state-party list:  ['National popular vote', 'Republican', 'Democratic', 'Republican', 'Republican', 'Republican', 'Democratic', 'Democratic', 'Democratic', 'Republican', 'Democratic', 'Democratic', 'Democratic', 'Democratic', 'Republican']


In [18]:
state_party_dataframe = pd.DataFrame(data = contents, columns = candidate_dataframe.columns)

print("Final state-party dataframe: \n")
state_party_dataframe.head()

Final state-party dataframe: 



,Year,1972,1976,1980,1984,1988,1992,1996,2000,2004,2008,2012,2016,2020,2024
0,National popular vote,Republican,Democratic,Republican,Republican,Republican,Democratic,Democratic,Democratic,Republican,Democratic,Democratic,Democratic,Democratic,Republican
1,Alabama,Republican,Democratic,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican
2,Alaska,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican,Republican
3,Arizona,Republican,Republican,Republican,Republican,Republican,Republican,Democratic,Republican,Republican,Republican,Republican,Republican,Democratic,Republican
4,Arkansas,Republican,Democratic,Republican,Republican,Republican,Democratic,Democratic,Republican,Republican,Republican,Republican,Republican,Republican,Republican


# 5. 💣 Export the DataFrame

⚓️ Create a reusable function to export a dataframe into a CSV file. ⤵️

In [19]:
def export_dataframe_to_csv(dataframe: pd.DataFrame, name: str) -> None:
    """
    Export the given dataframe into a CSV file with the given name.
    Parameters:
        - dataframe (pd.DataFrame): The dataframe to be exported.
        - name (str): The name of the CSV file.
    Returns:
        - None: Show the successful message with the exported location when the operation succeeds;
        otherwise, show an error message.
    """
    data_path = data.get_dataset_path("rbs", "processed", name, 1)

    try:
        dataframe.to_csv(data_path, index = False)
        print("🎉 Saved data to CSV at: ", data_path)
    except Exception as e:
        print(f"❌ Error: {e}")

⚓️ Export the clean 🇺🇸 US Presidential Elections by States dataframe. ⤵️

In [20]:
export_dataframe_to_csv(clean_states_dataframe, "clean_us_presidential_elections_by_states")

🎉 Saved data to CSV at:  /../../../../../../Volumes/Workstation/Datasets/Red.Blue.States/processed/clean_us_presidential_elections_by_states.csv


⚓️ Export the 🇺🇸 US Presidential Elections by States and party dataframe. ⤵️

In [21]:
export_dataframe_to_csv(state_party_dataframe, "us_presidential_elections_by_state_and_party")

🎉 Saved data to CSV at:  /../../../../../../Volumes/Workstation//Datasets/Red.Blue.States/processed/us_presidential_elections_by_state_and_party.csv


🎉 Congratulations! The `Data Cleaning` is complete!